In [1]:
%matplotlib inline 
import pandas as pd
import numpy as np
import xarray as xr
import os
import glob
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy.stats import norm
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import scipy.ndimage

import seaborn as sns
sns.set_style('white')
sns.set_context("notebook") #[poster, paper, talk, notebook]

## FUNCTIONS

In [2]:
def subregion_size(ds):
    dx = ds.x.max().values - ds.x.min().values
    dy = ds.y.max().values - ds.y.min().values
    return dx, dy


def plot_sd(ds, title, time=31, cmap='Blues', vmax=300):
    s = ds.isel(time=time)
    x_gt0 = s.snow.where(s.snow>0)
    x_gt0.plot.imshow(x='x', y='y', cmap=cmap, vmax=vmax, cbar_kwargs={'label': 'Snow Depth [cm]'})
    plt.tight_layout()

    
def kernel(n):
    # create mean filter 
    k = np.ones((n,n)) * 1./n**2
    return k

def smooth_over_grid(ds, k, time=31):
    s = ds.isel(time=time)
    smoothed = scipy.ndimage.convolve(s.snow, k)
    return smoothed

## DATA INPUTS

In [3]:
dx_input = 1000
flist = sorted(glob.glob('/Users/meganmason491/Documents/research/sierra/data/compiled_SUPERsnow_*{}m*'.format(dx_input))) #bsu
# flist=flist[0]

In [4]:
flist = [flist]

## MAIN SCRIPT

In [5]:
stns = ['DAN','SLI','TUM'] #could make dict so it's always connected to the file, but shortcut is to do sorted(glob.glob) -- ie always loads in same order :/
stns_lc = ['dan','sli','tum']
stns_longname = ['Dana Meadow', 'Slide Canyon', 'Tuolumne Meadow']

k = kernel(50)

def smoother(f):
    # input (f): file
    # returns (smoothed_ds): xarray dataframe with smoothed snow layer
    
    ds = xr.open_dataset(f)#; ds.close() #open lidar snow depth DataSet in Xarray -- ds
    smooth_holder = np.zeros((51, 335, 335))
    smoothed_ds = ds.copy()
    for t in range(51):
           
        #grab a single time step
        subds = ds.isel(time=t).snow
        # apply the mean smoother filter 
        smooth_holder[t,:,:] = scipy.ndimage.convolve(subds, k)
        
    # done looping through time 
    # append the smoothed numpy ndarray to the newly copied xarray dataframe
    smoothed_ds['smoothed_snow'] = (('time','y','x'), smooth_holder)
    return smoothed_ds

list_of_smoothed_datasets = []
for f in flist:
    list_of_smoothed_datasets.append(smoother(f))

#         smoothed = smooth_over_grid(ds, k)
#         smoothed
#         plot_sd(smoothed, title=stns_longname[i], vmax=300); plt.show()
#         plt.imshow(smoothed, cmap='RdBu', vmax=300)
#         plt.title(stns_longname[i])
#         plt.colorbar()
#         plt.show()
        
        #plot difference


AttributeError: 'list' object has no attribute 'tell'

In [ ]:
ds1, ds2, ds3 = list_of_smoothed_datasets
ds2

In [ ]:
# (smoothed_ds.smoothed_snow - smoothed_ds.snow).plot(x='x', y='y', col='time', col_wrap=10)
# smoothed_ds.smoothed_snow.plot(x='x', y='y', col='time', col_wrap=10)

print('Dana')
#raw
ds1_=ds1.isel(time=slice(1,51,10))
ds1_.snow.plot.imshow(x='x', y='y', col='time', col_wrap=5, cmap='Blues', vmax=250, cbar_kwargs={'label': 'Snow Depth [cm]'}); #plt.show()
# plt.savefig('../figs/dan_raw_ex.png', dpi=300)
#smoothed
ds1_=ds1.isel(time=slice(1,51,10))
ds1_.smoothed_snow.plot.imshow(x='x', y='y', col='time', col_wrap=5, cmap='Blues', vmax=250, cbar_kwargs={'label': 'Snow Depth [cm]'}); #plt.show()
# plt.savefig('../figs/dan_smooth_150m_ex.png', dpi=300)
#differenced
diff = ds1_.snow - ds1_.smoothed_snow
diff.plot.imshow(x='x', y='y', col='time', col_wrap=5, cmap='RdBu', vmax=250, cbar_kwargs={'label': 'Snow Depth [cm]'}); #plt.show()
# plt.savefig('../figs/dan_diff_ex.png', dpi=300)

# print('Slide')
# #raw
# ds2_=ds2.isel(time=slice(1,51,10))
# ds2_.snow.plot.imshow(x='x', y='y', col='time', col_wrap=5, cmap='Blues', vmax=250, cbar_kwargs={'label': 'Snow Depth [cm]'}); #plt.show()
# plt.savefig('../figs/sli_raw_ex.png', dpi=300)
# #smoothed
# ds2_=ds2.isel(time=slice(1,51,10))
# ds2_.smoothed_snow.plot.imshow(x='x', y='y', col='time', col_wrap=5, cmap='Blues', vmax=250, cbar_kwargs={'label': 'Snow Depth [cm]'}); #plt.show()
# plt.savefig('../figs/sli_smooth_150m_ex.png', dpi=300)
# #differenced
# diff = ds2_.snow - ds2_.smoothed_snow
# diff.plot.imshow(x='x', y='y', col='time', col_wrap=5, cmap='RdBu', vmax=250, cbar_kwargs={'label': 'Snow Depth [cm]'}); #plt.show()
# plt.savefig('../figs/sli_diff_ex.png', dpi=300)

# print('Tuolumne')
# #raw
# ds3_=ds3.sel(time=slice('2017-01-29','2017-06-04'))
# ds3_.snow.plot.imshow(x='x', y='y', col='time', col_wrap=5, cmap='Blues', vmax=250, cbar_kwargs={'label': 'Snow Depth [cm]'}); #plt.show()
# plt.savefig('../figs/tum_raw_ex.png', dpi=300)
# #smoothed
# ds3_=ds3.sel(time=slice('2017-01-29','2017-06-04'))
# ds3_.smoothed_snow.plot.imshow(x='x', y='y', col='time', col_wrap=5, cmap='Blues', vmax=250, cbar_kwargs={'label': 'Snow Depth [cm]'}); #plt.show()
# plt.savefig('../figs/tum_smooth_150m_ex.png', dpi=300)
# #differenced
# diff = ds3_.snow - ds3_.smoothed_snow
# diff.plot.imshow(x='x', y='y', col='time', col_wrap=5, cmap='RdBu', vmax=250, cbar_kwargs={'label': 'Snow Depth [cm]'}); #plt.show()
# plt.savefig('../figs/tum_diff_ex.png', dpi=300)

In [ ]:
# Single time example...
dx_input = 1000
flist = sorted(glob.glob('/Users/meganmason491/Documents/research/sierra/data/compiled_SUPERsnow_*{}m*'.format(dx_input))) #bsu
f = flist[0]
k = kernel(100)
ds = xr.open_dataset(f); ds.close()
snow_3m = ds.isel(time=31).snow
snow_smooth = scipy.ndimage.convolve(snow_3m, k)
snow_smooth

plt.imshow(snow_smooth, cmap='Blues')
plt.colorbar()




In [ ]:
plt.imshow(snow_3m, cmap='Blues', vmax=300)
plt.colorbar()



In [ ]:
plt.imshow(snow_3m - snow_smooth, cmap='RdBu'); plt.colorbar()

### TEST EVERYTHING

In [ ]:
ds

In [ ]:
ds.x.count().values

In [ ]:
stns = ['DAN','SLI','TUM'] #could make dict so it's always connected to the file, but shortcut is to do sorted(glob.glob) -- ie always loads in same order :/
stns_lc = ['dan','sli','tum']
stns_longname = ['Dana Meadow', 'Slide Canyon', 'Tuolumne Meadow']

k = kernel(50)

for i, f in enumerate(flist):
    
        print('Met Station Name:', stns_longname[i], '\n station ID:', stns[i])

        ds = xr.open_dataset(f); ds.close() #open lidar snow depth DataSet in Xarray -- ds

        print('x direction length {} [m]:'.format(ds.x.max().values - ds.x.min().values))
        print('y direction length {} [m]:'.format(ds.y.max().values - ds.y.min().values))

        dx, dy = subregion_size(ds) #subregion size -- used in figure naming

        plot_sd(ds, title=stns_longname[i], vmax=200);plt.show()